In [1]:
%load_ext autoreload
%autoreload 2

$$\begin{aligned}
&\min_X f(X) = \textrm{tr}(A^\top XB X^\top)  \\
& = \textrm{tr}(X^\top A^\top XB) & x = \textrm{vec}(X)\\
& = \left <\textrm{vec}(X),  \textrm{vec}(A^\top X B )  \right > \\
& = \left <\textrm{vec}(X), B^\top \otimes A^\top \cdot \textrm{vec}(X)  \right > \\ 
& = x^\top (B^\top \otimes A^\top) x\\ 
& = \left<AX, XB\right> \\
\mathbf{s.t.} & \\ 
&X \in \Pi_{n}
\end{aligned}$$

In [2]:
from qap_lp.models import *

In [3]:
import logging
import os
from logging.handlers import TimedRotatingFileHandler as TRFH

import pandas as pd

LOG_PATH = 'log'
FORMAT = '[%(name)s:%(levelname)s] [%(asctime)s] %(message)s'
logging.basicConfig(format=FORMAT)
if LOG_PATH is not None:
    if not os.path.exists(LOG_PATH):
        os.makedirs(LOG_PATH)
handler = TRFH(f'{LOG_PATH}/qap.log',
               when='M',
               interval=1,
               backupCount=7,
               encoding='utf8')
handler.setFormatter(logging.Formatter(FORMAT))
log = logging.getLogger()
log.addHandler(handler)
log.setLevel(logging.INFO)
log.handlers = [log.handlers[-1]]

In [11]:
instance_name = 'bur26a'

# mosek params
# msk_params = {**MSK_DEFAULT, **kwargs}

# coefficients
A0, B0 = parse(f'qapdata/{instance_name}.dat')
A = A0 / A0.max()
B = B0 / B0.max()
n, m = A.shape
e = np.ones(shape=n)
E = np.ones(shape=(n, n))
ab = np.kron(B.T, A.T)

# parse known solution
_, obj, arr = parse_sol(f'qapsoln/{instance_name}.sln')

param = QAPParam(A, B, n, m, e, E, ab, obj, arr)

## projection methods

working on problem:
### exact penalty

$$\begin{aligned}
 &\min_X f  + \mu\cdot | \textrm{tr}(XX^\top) -  n| \\
 &= f  + \mu\cdot n - \mu\cdot \textrm{tr}(XX^\top)  \\
\mathbf{s.t.} & \\
&Xe = X^\top e = e \\
&X \ge 0 \\
\end{aligned}$$

or

$$\begin{aligned}
& \min_X  f  + \mu\cdot n - \mu\cdot \textrm{tr}(XX^\top) - \Lambda \bullet X \\
\mathbf{s.t.} & \\
&Xe = X^\top e = e \\
\end{aligned}$$


very likely to become a concave function, cannot be directly solved by conic solver.



#### project point onto $D_n$

- projection may be computed by mosek
- or some other methods

#### derivatives

$$\begin{aligned}
& \nabla_X F_\mu  = A^\top XB + AXB^\top - 2\mu X \quad (-\Lambda)\\
& \nabla_\mu F_\mu  = n - \textrm{tr}(XX^\top) \\
& \nabla_\Lambda F_\mu  = - X
\end{aligned}$$

#### projected derivative

$$\begin{aligned}
&\min_D ||\nabla F_\mu + D ||_F^2  \\
\mathbf{s.t.} & \\
&D e = D^\top e = 0 \\ 
&D_{ij} \ge 0 \quad \textsf{if: } X_{ij} = 0\\
\end{aligned}$$


In [12]:
mu = 0.1

nabla = QAPDerivativeL2Penalty(param, mu)

def gt_min(x):
    return x[x >= -1e-4].min()

x = x0 = np.ones((n, n)) / n


alpha = alpha0 = 0.1

xol = param.xo
optimal = nabla.obj(xol)
print(nabla.obj(xol), nabla.original_obj(xol))
print(nabla.obj(x), nabla.original_obj(x))

13.42935979608503 13.42935979608503
17.187692098294942 14.687692098294942


#### gradient projection

- projection may be computed by mosek
- or some other methods

In [6]:
# dF = x
def pd_on_dc(param, dF, lb_indices, ub_indices=None, **kwargs):

    A, B, n, m, e, E, ab = param.A, param.B, param.n, param.m, param.e, param.E, param.ab

    model = mf.Model('projected_gradient_on_D_cone')
    D = model.variable("d", [*A.shape], dom.unbounded())
    v = model.variable(1, dom.greaterThan(0))
    m = expr.flatten(expr.add(D, dF))
   
    
    model.objective(mf.ObjectiveSense.Minimize, v)
    
    model.constraint(expr.sum(D, 0), dom.equalsTo(0))
    model.constraint(expr.sum(D, 1), dom.equalsTo(0))
    model.constraint(expr.vstack(v, m), dom.inQCone())
    constrs_lb = model.constraint(D.pick(*lb_indices), dom.equalsTo(0))
#     constrs_ub = model.constraint(D.pick(*ub_indices), dom.lessThan(0))

    # set params
    userCallback = set_mosek_model_params(model, **kwargs)
    model.setLogHandler(None)
    model.solve()

    model.flushSolutions()
    D_sol = D.level().reshape(A.shape)
    return D_sol, model, D, constrs_lb

In [7]:
def st(dp, x):
    model = mf.Model('step_size')
    v = model.variable(1, dom.greaterThan(0))
    V = expr.reshape(expr.repeat(v, n*n, 0), [n, n])
    delta = expr.mulElm(V, dp)
    model.constraint(expr.add(x, delta), dom.greaterThan(0))
    model.objective(mf.ObjectiveSense.Maximize, v)
#     model.setLogHandler(sys.stdout)
    try:
        model.solve()
        return v.level()[0]
    except:
        return 0

In [13]:
alpha = 0.1
logger = logging.getLogger('qap')

# gradient projection, not good for now
for i in range(1000):
    logger.info(f'====={i }====')
    
    #
    obj_old = nabla.obj(x)
    alpha *= 0.9
    d0 = nabla.partial_f(x) 
    
    # indices of active lower bound constraints 
    lb_x, lb_y = np.where(x <= 1e-4)
    lb_x = lb_x.tolist()
    lb_y = lb_y.tolist()
    
    dp, m, D, constrs_lb = pd_on_dc(
        param, d0, 
        (lb_x, lb_y),
    )
    ndf = np.abs(dp).max()
    
    stp = st(dp, x)
    
    logger.info(f"gradient norm: {ndf}")
    
    if stp <= 1e-6:
# 
#         try:
#             idx = constrs_lb.dual().argmax()
#             lb_x.pop(idx)
#             lb_y.pop(idx)
#         except Exception as e:
#             logging.exception(e)
#             lb_x, lb_y = np.where(x <= 1e-5)
#             lb_x = lb_x.tolist()
#             lb_y = lb_y.tolist()
#         logger.info(f"change active set @{i}")
#         dp, m, D, constrs_lb = pd_on_dc(
#             param, d0, 
#             (lb_x, lb_y)
#         )
        break
    
    objs = [(i, nabla.obj(x + i / 10 * stp * dp)) 
            for i in range(1, 11)]
    i_s, vs = min(objs, key=lambda x: x[-1])
    logger.info(f"steps: {i_s}, {vs}, {stp}")
    x = x + i_s / 10 * stp * dp
    logger.info(f"obj: {vs, vs - obj_old}, gap: {(vs - optimal)/optimal}")
    logger.info(f"trace deficiency: {n - x.dot(x.T).trace()}")

[(1, 1.451679059329539),
 (2, 1.4518583277130657),
 (3, 1.4522333003167445),
 (4, 1.4528039771405765),
 (5, 1.4535703581845618),
 (6, 1.4545324434486995),
 (7, 1.4556902329329908),
 (8, 1.457043726637435),
 (9, 1.4585929245620324),
 (10, 1.4603378267067828)]

In [10]:
obj_old

1.451695495166166

In [ ]:
x.round(4).tolist()

In [ ]:
stp

#### point proj

In [ ]:
def proj_on_dn(param, x, **kwargs):
    A, B, n, m, e, E, ab = param.A, param.B, param.n, param.m, param.e, param.E, param.ab
    
    model = mf.Model('projected_on_Dn')
    D = model.variable("d", [*A.shape], dom.inRange(0, 1))
    v = model.variable(1, dom.greaterThan(0.0))
    
    # m = vec(D - dF)
    m = expr.flatten(expr.sub(D, x))
    model.constraint(expr.sum(D, 0), dom.equalsTo(1))
    model.constraint(expr.sum(D, 1), dom.equalsTo(1))
    model.constraint(expr.vstack(v, m), dom.inQCone())
    model.objective(mf.ObjectiveSense.Minimize, v)

    # set params
    userCallback = set_mosek_model_params(model, **kwargs)
    model.setLogHandler(None)
    model.solve()

    model.flushSolutions()
    D_sol = D.level().reshape(A.shape)
    return D_sol

In [ ]:
def point_proj_naive():
    # point projection
    # - stuck at an corner point, vertex.
    # - once reached, it stays.
    for i in range(5):
        obj = nabla.obj(x, mu)
        print(f'====={i }====')
        alpha = 1/(i+1)
        d0 = nabla.partial_f(x, mu)
        x0 = x - d0 * alpha0
        xp = proj_on_dn(param, x0)
    #     # direction
    #     dx = xp - x
    #     st = gt_min(x/dx)
    #     print(f"stepsize: ", alpha, st)
    #     x -=  st * alpha * dx
        x = xp
        print(f"obj: ", nabla.original_obj(x), nabla.obj(x, mu), nabla.obj(x, mu) - obj)
        print(f"stopping: ", n - x.dot(x.T).trace())
        print(x.round(3).min())
        print(np.linalg.norm(d0))

In [ ]:
?constrs_lb

In [ ]:
idx = constrs_lb.dual().argmax()

In [ ]:
x.round(4)

In [ ]:
dp.round(4)

# eval

In [ ]:
x_int, _ = extract_sol_rounding(x, param.A, param.B)

In [ ]:
def check_obj_val(x_sol):
    _obj = A0.T.dot(x_sol).dot(B0).dot(x_sol.T).trace()
    print(f'original obj {_obj}')
    return _obj

In [ ]:
check_obj_val(x)

In [ ]:
nabla.original_obj(xol), nabla.obj(xol, mu)